In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
from sentence_transformers import SentenceTransformer
import numpy as np
import prince
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

'''
Variables: 
---------

corpus : list of documents
embeddings : documents embeddings of size NxM (N : number of documents, M : embedding dimension) 
red_emd : reduced embeddings matrix using dimentionality reduction
k : number of clusters
labels : documents labels
pred : list of clustering predicted clusters 

''';

In [14]:
def dim_red(mat, p):
    '''
    Perform dimensionality reduction

    Input:
    -----
        mat : NxM list 
        p : number of dimensions to keep 
    Output:
    ------
        red_mat : NxP list such that p<<m
    '''
    pca = prince.PCA(n_components=20)
    pca= pca.fit(pd.DataFrame(mat))
    reduction_dim = pca.transform(pd.DataFrame(mat))
    reduction_dim = reduction_dim.to_numpy()
    
    return reduction_dim

In [9]:
def clust(mat, k):
    '''
    Perform clustering

    Input:
    -----
        mat : input list 
        k : number of cluster
    Output:
    ------
        pred : list of predicted labels
    '''
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    result = model.fit(mat)

    return result.labels_

In [20]:
def stat_model(n_expr, mat, k, labels):
  means_nmi_score=0.
  varience_nmi_score=0.
  means_ari_score=0.
  varience_ari_score=0.

  results_nmi_score=[]
  results_ari_score=[]
  for i in range(n_expr):
    pred = clust(mat, k)
    results_nmi_score = normalized_mutual_info_score(pred,labels)
    results_ari_score = adjusted_rand_score(pred,labels)
  means_nmi_score = np.mean(results_nmi_score)
  means_ari_score = np.mean(results_ari_score)
  varience_nmi_score = np.std(results_nmi_score)
  varience_ari_score = np.std(results_ari_score)

  return means_nmi_score, varience_nmi_score, means_ari_score, varience_ari_score

In [5]:
# import data
ng20 = fetch_20newsgroups(subset='test')
corpus = ng20.data[:2000]
labels = ng20.target[:2000]
k = len(set(labels))

# embedding
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(corpus)

NMI: 0.03 
ARI: -0.00


In [15]:
# perform dimentionality reduction
red_emb = dim_red(embeddings, 20)

# perform clustering
pred = clust(red_emb, k)

# evaluate clustering results
nmi_score = normalized_mutual_info_score(pred,labels)
ari_score = adjusted_rand_score(pred,labels)

print(f'NMI: {nmi_score:.2f} \nARI: {ari_score:.2f}')

NMI: 0.40 
ARI: 0.23


In [21]:
n_expr=100
resultats = stat_model(n_expr, mat = red_emb, k = 20, labels = labels)

In [23]:
print(f"Moyenne nmi_score sur {n_expr} experience est:  {resultats[0]}")
print(f"Varience nmi_score sur {n_expr} experience est:  {resultats[1]}")
print(f"Moyenne ari_score sur {n_expr} experience est:  {resultats[2]}")
print(f"Moyenne ari_score sur {n_expr} experience est:  {resultats[3]}")

Moyenne nmi_score sur 100 experience est:  0.4154972492750745
Varience nmi_score sur 100 experience est:  0.0
Moyenne ari_score sur 100 experience est:  0.23239405767073365
Moyenne ari_score sur 100 experience est:  0.0
